# script.py

In [ ]:
from dotenv import load_dotenv
load_dotenv()  # Wczytaj zmienne środowiskowe z pliku .env, aby móc korzystać z ukrytych kluczy API.

# Definicja dwóch wariantów promptów do testowania generowania nazw produktów
prompt_A = """Opis produktu: Buty, które pasują dla każdego rozmiaru stopy.
Słowa początkowe: dostosowanie, dopasowanie, uniwersalny rozmiar
Nazwy produktów:"""

prompt_B = """Opis produktu: Automat do przygotowania mlecznych koktajli w domu.
Słowa początkowe: szybki, zdrowy, kompaktowy.
Nazwy produktów: HomeShaker, Fit Shaker, QuickShake, Shake Maker
Opis produktu: Zegarek, który podaje dokładny czas w przestrzeni kosmicznej
Słowa początkowe: astronauta, odporne na przestrzeń kosmiczną, orbita eliptyczna
Nazwy produktów: AstroTime, SpaceGuard, Orbit-Accurate, EliptoTime.
Opis produktu: Buty, które pasują dla każdego rozmiaru stopy.
Słowa początkowe: dostosowanie, dopasowanie, uniwersalny rozmiar
Nazwy produktów:"""

# Lista testowanych promptów
test_prompts = [prompt_A, prompt_B]

import pandas as pd  # Biblioteka do obsługi danych tabelarycznych
from openai import OpenAI  # Import klienta OpenAI do obsługi API
import os  # Moduł do zarządzania zmiennymi środowiskowymi

# Tworzenie klienta OpenAI z kluczem API przechowywanym w zmiennej środowiskowej
client = OpenAI(
    api_key=os.environ['OPENAI_API_KEY'],  # Pobranie klucza API z systemowych zmiennych środowiskowych
)

def get_response(prompt):
    """Funkcja do uzyskania odpowiedzi z modelu GPT na podany prompt."""
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # Model używany do generowania odpowiedzi
        messages=[
            {"role": "system", "content": "Jesteś pomocnym asystentem."},  # Kontekst dla modelu
            {"role": "user", "content": prompt}  # Właściwy prompt przekazany do modelu
        ]
    )
    return response.choices[0].message.content  # Zwrócenie treści pierwszej wygenerowanej odpowiedzi

responses = []  # Lista na zebrane odpowiedzi
num_tests = 5  # Liczba testów dla każdego prompta

for idx, prompt in enumerate(test_prompts):  # Iteracja po promptach
    var_name = chr(ord('A') + idx)  # Zamiana indeksu na literę (A, B, C...)
    
    for i in range(num_tests):  # Wykonanie testów wielokrotnie dla lepszego porównania
        response = get_response(prompt)  # Pobranie odpowiedzi od modelu
        data = {
            "variant": var_name,  # Oznaczenie wariantu (A/B)
            "prompt": prompt,  # Treść prompta
            "response": response  # Odpowiedź modelu
        }
        responses.append(data)  # Dodanie wyniku do listy

# Konwersja wyników na obiekt DataFrame (tabelę Pandas)
df = pd.DataFrame(responses)

# Zapis wyników do pliku CSV
df.to_csv("odpowiedzi.csv", index=False)

# Wyświetlenie tabeli w terminalu
print(df)

"""
Opis działania kodu:
Ten skrypt testuje dwa różne warianty promptów do generowania nazw produktów przy użyciu modelu GPT-3.5-turbo.
Najpierw wczytywane są zmienne środowiskowe, w tym klucz OpenAI API. Następnie definiowane są dwa prompty - jeden 
z minimalną ilością przykładów, a drugi z większą liczbą przykładów. Każdy prompt jest testowany pięć razy 
(z możliwością łatwego zwiększenia liczby testów). Wyniki są zbierane w obiekcie Pandas DataFrame i zapisywane 
do pliku CSV, co pozwala na późniejszą analizę.

Zastosowanie:
- Porównanie skuteczności różnych wariantów promptów.
- Analiza jakości generowanych odpowiedzi.
- Tworzenie benchmarków dla różnych modeli AI.
"""

# script2.py

In [1]:
from dotenv import load_dotenv
load_dotenv() # wczytaj zmienne środowiskowe z pliku .env.

import ipywidgets as widgets
from IPython.display import display 
import pandas as pd
# Wczytaj plik odpowiedzi.csv
df = pd.read_csv("odpowiedzi.csv") # Wymieszaj ramkę danych
df = df.sample(frac=1).reset_index(drop=True)
# df to Twoja ramka danych, a response to kolumna zawierająca numer tekstu, który chcesz sprawdzić
response_index = 0
# dodaj nową kolumnę w celu przechowania informacji zwrotnej
df['feedback'] = pd.Series(dtype='str')
def on_button_clicked(b):
    global response_index
    # podmień kciuki w górę i w dół na jedynki i zera
    user_feedback = 1 if b.description == "\U0001F44D" else 0
    # zaktalizuj kolumnę feedback 
    df.at[response_index, 'feedback'] = user_feedback
    response_index += 1
    if response_index < len(df):
        update_response() 
    else:
        # zapisz informację zwrotną do pliku CSV
        df.to_csv("results.csv", index=False)
        print("Zakończono testy A/B. Oto wyniki: ") 
        # Policz wynik i liczbę wierszy dla każdego z wariantów 
        summary_df = df.groupby('variant').agg(
                count=('feedback', 'count'),
                score=('feedback', 'mean')).reset_index()
        print(summary_df)
def update_response():
    new_response = df.iloc[response_index]['response'] 
    if pd.notna(new_response):
        new_response = "<p>" + new_response + "</p>" 
    else:
        new_response = "<p>Brak odpowiedzi</p>"
    response.value = new_response
    count_label.value = f"Odpowiedź: {response_index + 1}"
    count_label.value += f"/{len(df)}"
response = widgets.HTML()
count_label = widgets.Label()
update_response()
thumbs_up_button = widgets.Button(description='\U0001F44D')
thumbs_up_button.on_click(on_button_clicked)
thumbs_down_button = widgets.Button(description='\U0001F44E')
thumbs_down_button.on_click(on_button_clicked)
button_box = widgets.HBox([thumbs_down_button, thumbs_up_button])
display(response, button_box, count_label)

HTML(value='<p>1. UniFitShoes\n2. PerfectFit Footwear\n3. AllSize Adapt Shoes\n4. UniversalStep Sneakers\n5. I…

Label(value='Odpowiedź: 1/10')